In [ ]:
import requests
import time
import pandas as pd
from config import email, key
import matplotlib.pyplot as plt
import json

In [ ]:
events = ["ci37908735","nc72737985","ci37374687","nc72282711","ci15200401","ci15199681","ci10736069",
          "ci14408052","ci14383980","nc40216664","nc40204628"]

i=0

earthquakes_list = []

for event in events:
    base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
    times_url = "&starttime=1990-01-01&endtime=2019-06-15"
    magnitude_url ="&minmagnitude=5.0"
    #geometry_url = "&minlatitude=32.43&maxlatitude=42.05&minlongitude=-124.28&maxlongitude=-114.11"
    eventid_url = f"&eventid={event}"
    
    search = requests.get(base_url + times_url + magnitude_url + eventid_url)
    
    response = search.json()

    identifier = response['id']
    mag = response['properties']['mag']
    url = response['properties']['url']
    epochtime = response['properties']['time']
    four_weeks_before = epochtime - 2419200000
    four_weeks_after = epochtime + 2419200000
    event_date= time.strftime('%Y%m%d',  time.gmtime(epochtime/1000))
    month_before= time.strftime('%Y%m%d',  time.gmtime(four_weeks_before/1000))
    month_after = time.strftime('%Y%m%d',  time.gmtime(four_weeks_after/1000))
    converted_time = time.strftime('%H:%M:%S',  time.gmtime(epochtime/1000))
    timezone=response['properties']['tz']
    place = response['properties']['place']
    significance=response['properties']['sig']
    lon = response['geometry']['coordinates'][0]
    lat = response['geometry']['coordinates'][1]
    depth = response['geometry']['coordinates'][2]
    record_dict = {"Identifier":identifier, "Location":place, "Magnitude":mag, "Event Date":event_date,
                   "Month Before":month_before, "Month After":month_after, "Time":converted_time,"Time Zone":timezone, 
                   "Significance":significance, "Lat":lat, "Lon":lon, "Depth":depth, "Epoch Time":epochtime}
    earthquakes_list.append(record_dict)
        
    i += 1
          
earthquakes_df = pd.DataFrame(earthquakes_list)

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Mexico")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("MX")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Nevada")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("NV")].reset_index()

earthquakes_df = earthquakes_df.fillna(value="")

earthquakes_abbrev =earthquakes_df[['Location', 'Event Date', 'Magnitude']]

pd.set_option('display.max_rows', None)

earthquakes_df 

In [ ]:
earthquakes_df.to_csv("earthquake_list.csv")

In [ ]:
earthquake = 0
record_list = []

for rows in earthquakes_df.iterrows():

    # base_url finds air quality readings by Box

    base_url = "https://aqs.epa.gov/data/api/dailyData/byBox?"

    #parameters for byBox for CRITERIA Pollutants 
    #42401="Sulfur Dioxide" 
    #42602="Nitrogen Dioxide"
    #42201="Ozone"
    #42101 = "Carbon Monoxide"

    begin = earthquakes_df.iloc[earthquake, 5]
    end = earthquakes_df.iloc[earthquake, 6]
    minlat=earthquakes_df.iloc[earthquake,10]-1
    maxlat=earthquakes_df.iloc[earthquake,10]+1
    minlon=earthquakes_df.iloc[earthquake,11]-1
    maxlon=earthquakes_df.iloc[earthquake,11]+1
    earthquake_location=earthquakes_df.iloc[earthquake,2]
    event_date =earthquakes_df.iloc[earthquake, 4]
    
    param_url = f"param=42401,42602,44201,42101,81102,88101"
    date_range_url = f"&bdate={begin}&edate={end}"
    geometry_url = f"&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}"

    url = (base_url + param_url + "&email=" + email + "&key=" + key + date_range_url + geometry_url)
    
    search = requests.get(url)
    responses = search.json()
    
    i = 0
    
    for record in responses['Data']:
        site = responses['Data'][i]['local_site_name']
        station = responses['Data'][i]['site_number']
        parameter = responses['Data'][i]['parameter']
        date = responses['Data'][i]['date_local']
        measurement = responses['Data'][i]['arithmetic_mean']
        measurement_unit = responses['Data'][i]['units_of_measure']
        event_type=responses['Data'][i]["event_type"]
        lon = responses['Data'][i]['longitude']
        lat = responses['Data'][i]['latitude']
        earthquake_id = earthquakes_df.iloc[earthquake, 1]
        record_dict = {"Earthquake ID":earthquake_id,"Parameter":parameter,"Date":date,"Station":station,"Site":site, 
                       "Lat":lat,"Lon":lon, "Measurement":measurement, "Unit":measurement_unit,"Event Type":event_type}
        record_list.append(record_dict)

        i += 1
    
    earthquake += 1    



In [ ]:
earthquake

In [ ]:
record_df = pd.DataFrame(record_list)
pd.set_option('display.max_rows', 100)
record_df 

In [ ]:
param_summary = record_df[['Parameter', "Measurement", "Unit"]]
param_min = param_summary.groupby(['Parameter']).min()
param_max = param_summary.groupby(['Parameter']).max()
param_mean = param_summary.groupby(['Parameter']).mean()
param_median = param_summary.groupby(['Parameter']).median()
param_count = param_summary.groupby(['Parameter']).count()
stats = pd.merge(param_min, param_max, on = "Parameter")
stats = stats.rename(columns = {"Measurement_x":"Min", "Measurement_y":"Max","Unit_x":"Unit"})
stats = stats.drop(columns = "Unit_y")
stats = pd.merge(stats, param_mean, on= 'Parameter')
stats = stats.rename(columns ={'Measurement':"Mean"})
stats = pd.merge(stats, param_median, on = 'Parameter')
stats = stats.rename(columns ={'Measurement':'Median'})
stats = pd.merge(stats, param_count, on = 'Parameter')
stats = stats.rename(columns = {'Measurement':'Count','Unit_x':"Unit"})
stats = stats.drop(columns = "Unit_y")
stats = stats[['Unit','Count','Min','Max','Mean','Median']]
stats.to_csv('Stats_California11.csv')
stats

In [ ]:
# establish dataframe for each earthquake

In [ ]:

events = ["ci37908735","nc72737985","ci37374687","nc72282711","ci15200401","ci15199681","ci10736069",
          "ci14408052","ci14383980","nc40216664","nc40204628"]

df_eq0 = record_df[record_df['Earthquake ID']==events[0]]
df_eq1 = record_df[record_df['Earthquake ID']==events[1]]
df_eq2 = record_df[record_df['Earthquake ID']==events[2]]
df_eq3 = record_df[record_df['Earthquake ID']==events[3]]
df_eq4 = record_df[record_df['Earthquake ID']==events[4]]
df_eq5 = record_df[record_df['Earthquake ID']==events[5]]
df_eq6 = record_df[record_df['Earthquake ID']==events[6]]
df_eq7 = record_df[record_df['Earthquake ID']==events[7]]
df_eq8 = record_df[record_df['Earthquake ID']==events[8]]
df_eq9 = record_df[record_df['Earthquake ID']==events[9]]
df_eq10 = record_df[record_df['Earthquake ID']==events[10]]
                     
earthquake_records = ['df_eq0', 'df_eq1', 'df_eq2', 'df_eq3', 'df_eq4', 'df_eq5', 
                      'df_eq6', 'df_eq7', 'df_eq8', 'df_eq9', 'df_eq10']

parameters = ['Carbon monoxide', 'Nitrogen dioxide (NO2)', 'Ozone', 
              'PM10 Total 0-10um STP', 'PM2.5 - Local Conditions', 'Sulfur dioxide']



In [ ]:
recordgroupby_df = record_df.groupby(['Earthquake ID','Parameter','Date'])

# giant_pivot = pd.pivot_table(recordgroupby_df, values=['Measurement'],columns = ['Parameter'], index = ['Earthquake ID','Date'])

#giant#_pivot = pd.pivot_table(recordgroupby_df, values=['Measurement'],columns = ['Parameter'], index = ['Earthquake ID','Date'])


# giant_pivot.to_csv('GiantPivot.csv')

pd.set_option('display.max_rows', None)

#giant_pivot
recordgroupby_df.head()


In [ ]:
skinny_df = recordgroupby_df['Earthquake ID','Parameter','Date','Measurement']
skinny_df.head(5)

In [ ]:
events = ["ci37908735","nc72737985","ci37374687","nc72282711","ci15200401","ci15199681","ci10736069",
          "ci14408052","ci14383980","nc40216664","nc40204628"]

parameters = ['Carbon monoxide', 'Nitrogen dioxide (NO2)', 'Ozone', 
              'PM10 Total 0-10um STP', 'PM2.5 - Local Conditions', 'Sulfur dioxide']

earthquake = 0
parameter = 2
event = 'ci10736069'

epochtime = earthquakes_df.iloc[earthquake, 13]
aqi_event_date = time.strftime('%Y-%m-%d',  time.gmtime(epochtime/1000))
earthquake_location=earthquakes_df.iloc[earthquake,2]
earthquake_id = earthquakes_df.iloc[earthquake, 1]

y_values = record_df[record_df['Parameter']=='Carbon monoxide']
y_values = y_values[y_values['Earthquake ID']==events[6]]
y_values = y_values[y_values['Station']=='0005']
y_values = y_values[['Date','Measurement']].sort_values('Date')
# y_values = y_values[y_values['Date','Measurement']]


#df_eq0 = record_df[record_df['Earthquake ID']==events[0]]

#x_values = giant_pivot.loc[['ci10736069','Date']].tolist()

y_values

# plt.plot(record_NO2['Date'],record_NO2['Measurement'])
# plt.title('{0}'.format("NO2:" + earthquake_location + " " + aqi_event_date), fontsize =30)
# plt.xlabel("Date", fontsize = 18)
# plt.ylabel("NO2 (parts per billion)", fontsize =18)
# plt.rcParams["figure.figsize"] = [16,8]
# plt.rcParams['xtick.labelsize']=11
# plt.rcParams['ytick.labelsize']=16
# plt.xticks(rotation=90)
# plt.axvline(x=aqi_event_date, color='r', linewidth=4)
# plt.grid()
# plt.show()


In [ ]:
y_values_grouped = y_values.groupby(['Date']).mean()
y_values_grouped

In [ ]:
y_values_grouped.plot.line()